<a href="https://colab.research.google.com/github/iffyaiyan/LSTM/blob/main/LSTM_for_Part_of_Speech_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# importing the necessary libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline


In [19]:
training_data = [
    ("The cat ate the cheese".lower().split(), ["DET", "NN", "V", "DET", "NN"]),
    ("She read that book".lower().split(), ["NN", "V", "DET", "NN"]),
    ("The dog loves art".lower().split(), ["DET", "NN", "V", "NN"]),
    ("The elephant answers the phone".lower().split(), ["DET", "NN", "V", "DET", "NN"])
]

# Dictionay to map word to indices
word2idx = {}
for sent, tags in training_data:
  for word in sent:
    if word not in word2idx:
      word2idx[word] = len(word2idx)

# Create a dictionary that maps tags to indices
tag2idx = {"DET": 0, "NN": 1, "V": 2}

In [20]:
#Printing the dictionart created
print(word2idx)

{'the': 0, 'cat': 1, 'ate': 2, 'cheese': 3, 'she': 4, 'read': 5, 'that': 6, 'book': 7, 'dog': 8, 'loves': 9, 'art': 10, 'elephant': 11, 'answers': 12, 'phone': 13}


We can see each unique word has unique index assosciated with it

In [21]:
import numpy as np

#Creating a helper function to convert a sequence of words to a Tensor of numerical values

def prepare_sequence(seq, to_idx):
  '''
    seq: sequence of words
    to_idx: dictionary to index 
  '''
  idxs = [to_idx[w] for w in seq]
  idxs = np.array(idxs)
  return torch.from_numpy(idxs)

In [22]:
# Lets use the above defined function

example = prepare_sequence("She ate the Cheese cat loves the".lower().split(), word2idx)
print(example)

tensor([4, 2, 0, 3, 1, 9, 0])


In [23]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        
        super(LSTMTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

        self.hidden = self.init_hidden()

        
    def init_hidden(self):

        return (torch.zeros(1, 1, self.hidden_dim),
                torch.zeros(1, 1, self.hidden_dim))
    
    def forward(self, sentence):

        embeds = self.word_embeddings(sentence)

        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)

        tag_outputs = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_outputs, dim=1)
        
        return tag_scores

#Define how the model trains

In [24]:

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word2idx), len(tag2idx))

loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [25]:
test_sentence = "Elephant loves the book cheese".lower().split()

inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)
_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n', predicted_tags)

tensor([[-1.2237, -1.2054, -0.9008],
        [-1.2353, -1.2004, -0.8961],
        [-1.1850, -1.1714, -0.9563],
        [-1.3922, -1.0608, -0.9031],
        [-1.3229, -1.0724, -0.9379]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([2, 2, 2, 2, 2])


#Train the Model

In [26]:
n_epochs = 300

for epoch in range(n_epochs):

  epoch_loss = 0.0

  for sentence, tags in training_data:

    model.zero_grad()

    model.hidden = model.init_hidden()

    sentence_in = prepare_sequence(sentence, word2idx)
    targets = prepare_sequence(tags, tag2idx)

    tag_scores = model(sentence_in)

    loss = loss_function(tag_scores, targets)
    epoch_loss += loss.item()
    loss.backward()

    optimizer.step()

  if(epoch%20 == 19):
    print("Epoch: %d, loss: %1.5f" % (epoch+1, epoch_loss/len(training_data)))

Epoch: 20, loss: 0.96931
Epoch: 40, loss: 0.80035
Epoch: 60, loss: 0.61774
Epoch: 80, loss: 0.51589
Epoch: 100, loss: 0.45520
Epoch: 120, loss: 0.38913
Epoch: 140, loss: 0.30337
Epoch: 160, loss: 0.20163
Epoch: 180, loss: 0.11180
Epoch: 200, loss: 0.06761
Epoch: 220, loss: 0.04700
Epoch: 240, loss: 0.03563
Epoch: 260, loss: 0.02850
Epoch: 280, loss: 0.02364
Epoch: 300, loss: 0.02011


In [27]:
test_sentence = "Elephant loves the book cheese".lower().split()

inputs = prepare_sequence(test_sentence, word2idx)
inputs = inputs
tag_scores = model(inputs)
print(tag_scores)

_, predicted_tags = torch.max(tag_scores, 1)
print('\n')
print('Predicted tags: \n', predicted_tags)

tensor([[-4.5045e+00, -2.9505e+00, -6.5472e-02],
        [-5.4710e+00, -3.7293e+00, -2.8623e-02],
        [-8.6447e-03, -5.0243e+00, -6.1991e+00],
        [-5.8634e+00, -6.1005e-03, -5.7321e+00],
        [-4.8022e+00, -1.1387e-01, -2.3085e+00]], grad_fn=<LogSoftmaxBackward>)


Predicted tags: 
 tensor([2, 2, 0, 1, 1])
